In [1]:
from tqdm import tqdm
import os
import jax.numpy as jnp
import jax.random as random
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
import arviz as az
from scipy.special import gammaln, log1p, expit
from pathlib import Path
import re, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.stats     import gaussian_kde
from scipy.integrate import trapezoid
from scipy.interpolate import interp1d
from typing import Dict, Any



rng = np.random.default_rng(42)

## Utilities for plotting and saving beta densities

In [3]:
# Save beta density histograms as JPEG files
def save_beta_density_hist(
        idata,
        *,
        output_folder: str | Path,
        experiment: str,
        regex: str = r"beta_\d+",
        bins: int = 40,
        dpi: int = 300,
):
    """
    One figure per beta variable (KDE + histogram) written to
    {output_folder}/{experiment}_{varname}.jpg.
    Histogram now shows *frequency* and the x-axis label is removed.
    """
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)

    posterior = idata.posterior
    var_names = [v for v in posterior.data_vars if re.fullmatch(regex, v)]
    if not var_names:
        raise ValueError(f"No variables match pattern {regex!r}")

    for var in var_names:
        draws = posterior[var].values.reshape(-1)

        fig, (ax_kde, ax_hist) = plt.subplots(
            2, 1,
            figsize=(6, 4),
            sharex=True,
            gridspec_kw={"height_ratios": (2, 1)},
        )

        # KDE (top)
        az.plot_kde(draws, ax=ax_kde, plot_kwargs={"color": "steelblue"})
        ax_kde.set_ylabel("Density")
        ax_kde.set_title("")  # no title
        ax_kde.set_xlabel("")  # remove x-axis label
        ax_kde.grid(True)

        # Histogram (bottom) — raw counts
        ax_hist.hist(draws, bins=bins, density=False,
                     color="steelblue", alpha=0.6)
        ax_hist.set_ylabel("Frequency")
        ax_hist.set_xlabel("")  # remove x-axis label
        ax_hist.grid(True)

        fig.tight_layout()

        fname = output_folder / f"{experiment}_{var}.jpg"
        fig.savefig(fname, dpi=dpi, bbox_inches="tight")
        plt.close(fig)
    print(f"Saved experiment {experiment} beta densities to {output_folder}")


# Plot empirical prior vs posterior KDEs
def plot_df_prior_vs_posterior(
        df_sim,
        idata,
        *,
        kde_bw: str | float | None = None,  # bandwidth for gaussian_kde
        density_cut: float = 1e-3,  # threshold for x-axis limits
        xgrid_len: int = 2000,
        save_to: str | Path | None = None,
):
    """
    Overlay empirical prior (from profile-likelihoods) with posterior KDEs.

    Parameters
    ----------
    df_sim : DataFrame
        Columns ['source', 'outcome', 'point', 'value'];  'value' is log-lik.
    idata  : arviz.InferenceData
        Must contain variables  beta_<s>.
    kde_bw : str | float | None
        Passed to gaussian_kde(bw_method=...).  None → Scott’s rule.
    density_cut : float
        x-axis is restricted to points where max(prior, posterior) > density_cut.
    xgrid_len : int
        Number of points used to draw the KDE curve.
    """
    import re
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.integrate import trapezoid
    from scipy.stats import gaussian_kde

    # ── 1) detect β_s in the posterior ───────────────────────────
    beta_vars = [v for v in idata.posterior.data_vars
                 if re.fullmatch(r"beta_\d+", v)]
    sources_avl = sorted(int(v.split("_")[1]) for v in beta_vars)
    if not sources_avl:
        raise ValueError("No beta_* variables found in idata.posterior")

    # ── 2) prepare empirical priors ──────────────────────────────
    prior_dens = {}
    for s in sources_avl:
        sub = df_sim[df_sim["source"] == s]
        g = sub.groupby("point")["value"].sum().sort_index()
        x = g.index.to_numpy()
        ll = g.values - g.values.max()
        p = np.exp(ll)
        p = p / trapezoid(p, x)  # normalise
        prior_dens[s] = (x, p)

    # ── 3) KDEs for posterior samples ────────────────────────────
    post_kde = {}
    sample_rng = {}
    for s in sources_avl:
        samples = idata.posterior[f"beta_{s}"].values.flatten()
        post_kde[s] = gaussian_kde(samples, bw_method=kde_bw)
        sample_rng[s] = (samples.min(), samples.max())

    # ── 4) plotting ──────────────────────────────────────────────
    n = len(sources_avl)
    ncols, nrows = 2, int(np.ceil(n / 2))
    fig, axes = plt.subplots(nrows, ncols, figsize=(10, 4 * nrows))
    axes = axes.flatten()

    for idx, s in enumerate(sources_avl):
        x_prior, p_prior = prior_dens[s]
        xmin = min(x_prior.min(), sample_rng[s][0])
        xmax = max(x_prior.max(), sample_rng[s][1])
        xgrid = np.linspace(xmin, xmax, xgrid_len)

        p_post = post_kde[s](xgrid)

        # ---- x-axis trimming by density_cut ---------------------
        mask = np.maximum(
            np.interp(xgrid, x_prior, p_prior, left=0, right=0),
            p_post
        ) > density_cut

        if mask.any():
            x_lim = (xgrid[mask].min(), xgrid[mask].max())
        else:  # fallback: full span
            x_lim = (xmin, xmax)

        # ---- draw curves ---------------------------------------
        axes[idx].plot(x_prior, p_prior, label="Prior (Interpolated)")
        axes[idx].plot(xgrid, p_post, label="Posterior (KDE)")

        axes[idx].set_title(f"Source {s}")
        axes[idx].set_xlabel(rf"$\beta_{{{s}}}$")
        axes[idx].set_ylabel("Density")
        axes[idx].grid(True)
        axes[idx].legend()
        axes[idx].set_xlim(*x_lim)

    # hide unused axes
    for ax in axes[n:]:
        ax.axis("off")

    fig.tight_layout()
    save_to
    if save_to is not None:
        fig.savefig(save_to, dpi=300, bbox_inches="tight")
    else:
        plt.show()






# KL divergence between prior and posterior densities
def kl_prior_posterior_beta(
    df_sim,
    idata,
    *,
    posterior_bins: int | None = None,
    eps: float = 1e-12,
):
    """
    Compute KL( posterior || prior) for each beta_s parameter.

    The prior density is taken from the profile-likelihood table `df_sim`
    (already normalised to integrate to 1).
    The posterior density is estimated with a histogram that uses exactly
    the same support grid, so the numerical integration is straightforward.

    Parameters
    ----------
    df_sim : DataFrame
        Columns: ['source','outcome','point','value'] with log-likelihood
        values per grid point.
    idata : arviz.InferenceData
        Must contain posterior variables  beta_<s>.
    posterior_bins : int, optional
        If given, overrides the grid from `df_sim` and builds an equally
        spaced grid with this many bins.  `None` (default) means: “use the
        grid contained in df_sim”.
    eps : float
        Small positive number added to both densities to avoid log(0).

    Returns
    -------
    kl_dict : dict  {source: KL divergence}
    """
    import re
    import numpy as np
    from scipy.integrate import trapezoid

    # -------- 1) Which β_s parameters are present?
    beta_vars   = [v for v in idata.posterior.data_vars if re.fullmatch(r"beta_\d+", v)]
    sources_avl = sorted(int(v.split("_")[1]) for v in beta_vars)
    if not sources_avl:
        raise ValueError("No beta_* variables found in idata.posterior")

    kl_dict = {}

    for s in sources_avl:
        # ---------- prior density on its grid --------------------
        sub = df_sim[df_sim["source"] == s]
        g   = sub.groupby("point")["value"].sum().sort_index()
        x_prior = g.index.to_numpy()
        ll = g.values - g.values.max()
        p_prior = np.exp(ll)
        p_prior = p_prior / trapezoid(p_prior, x_prior)

        # ---------- choose the grid / bin edges ------------------
        if posterior_bins is None:
            # use the df_sim grid
            # build edges so that each bin is centred on a prior grid point
            dx           = np.diff(x_prior)
            left_edges   = x_prior[:-1] - dx / 2.0
            right_edges  = x_prior[:-1] + dx / 2.0
            edges        = np.concatenate(
                ([left_edges[0]], right_edges, [x_prior[-1] + dx[-1] / 2.0])
            )
        else:
            edges = np.linspace(x_prior.min(), x_prior.max(), posterior_bins + 1)

        # ---------- posterior density on the same grid -----------
        samples = idata.posterior[f"beta_{s}"].values.flatten()
        counts, _ = np.histogram(samples, bins=edges, density=True)
        # `counts` is already a density (area = 1); align it with mid-points
        if posterior_bins is None:
            p_post = counts
            dx = np.diff(edges)                # width per bin
        else:
            # re-evaluate prior on the new grid to match dimensions
            mid   = 0.5 * (edges[:-1] + edges[1:])
            from scipy.interpolate import interp1d
            interp = interp1d(x_prior, p_prior, bounds_error=False, fill_value=0.0)
            p_prior = interp(mid)
            p_post  = counts
            dx      = np.diff(edges)

        # ---------- KL divergence (numerical integral) -----------
        # add eps to avoid division by / log of zero
        p_post_safe  = p_post  + eps
        p_prior_safe = p_prior + eps
        kl = np.sum(dx * p_post_safe * np.log(p_post_safe / p_prior_safe))

        kl_dict[s] = kl

    return kl_dict

# Summarise prior and posterior statistics for beta parameters
def summarise_beta_prior_posterior(
        df_sim,
        idata,
        kl_vals,
        experiment_label: int = 1
):
    """
    For each beta_s parameter:

        • numerical mean  & median of the PRIOR density
        • mean, median and 0.95 CI of the POSTERIOR draws
        • flag indicating whether the CI contains the prior mean

    Returns
    -------
    stats_df : pandas.DataFrame
        Columns
            source
            prior_mean   prior_median
            post_mean    post_median
            post_ci_lower  post_ci_upper
            ci_covers_prior_mean   (bool)
    """

    # 1 ── locate the beta variables in the posterior
    beta_vars = [v for v in idata.posterior.data_vars if re.fullmatch(r"beta_\d+", v)]

    sources_avl = sorted(int(v.split("_")[1]) for v in beta_vars)
    if not sources_avl:
        raise ValueError("No beta_* variables found in idata.posterior")

    records = []

    # 2 ── loop over sources
    for s in sources_avl:
        # ----- PRIOR density on grid ----------------------------
        sub = df_sim[df_sim["source"] == s]
        g = sub.groupby("point")["value"].sum().sort_index()
        x = g.index.to_numpy()
        ll = g.values - g.values.max()  # stabilise exponentiation
        p = np.exp(ll)
        p = p / trapezoid(p, x)  # normalise

        prior_mean = trapezoid(x * p, x)

        # median: CDF crosses 0.5
        # cumulative integral (trapezoidal)
        cdf = np.concatenate(
            ([0.0],
             np.cumsum((p[:-1] + p[1:]) / 2 * np.diff(x)))
        )
        # interpolate to find the 0.5 point
        median_interp = interp1d(cdf, x, bounds_error=False)
        prior_median = float(median_interp(0.5))

        # ----- POSTERIOR summary --------------------------------
        samples = idata.posterior[f"beta_{s}"].values.flatten()
        post_mean = float(np.mean(samples))
        post_median = float(np.median(samples))
        ci_lower, ci_upper = np.percentile(samples, [2.5, 97.5])

        covers = (ci_lower <= prior_mean) and (prior_mean <= ci_upper)
        mean_error = prior_mean - post_mean
        median_error = prior_median - post_median

        records.append(dict(
            experiment = experiment_label,
            source="beta{}".format(s),
            prior_mean=prior_mean,
            prior_median=prior_median,
            post_mean=post_mean,
            post_median=post_median,
            mean_error=mean_error,
            median_error=median_error,
            kl_divergence = kl_vals.get(s, np.nan),
            post_ci_lower=ci_lower,
            post_ci_upper=ci_upper,
            ci_covers_prior_mean=covers,
        ))

    stats_df = pd.DataFrame.from_records(records)
    return stats_df

## Utilities using sample

In [178]:
def plot_df_prior_vs_posterior(
    idata,
    beta_df,
    *,
    kde_bw: str | float | None = None,   # gaussian_kde bandwidth
    density_cut: float = 1e-3,           # trim x-axis where both dens < cut
    xgrid_len: int = 2_000,
    save_to: str | Path | None = None,
):
    """
    Overlay prior and posterior densities for every β_s found in *idata*.

    Parameters
    ----------
    idata : arviz.InferenceData
        Must contain variables  beta_<s>.
    prior_samples : dict[int, ndarray]
        Mapping  {source s → 1-D array of prior β draws}.
    kde_bw, density_cut, xgrid_len, save_to
        Same meaning as before.
    """
    import re
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.stats import gaussian_kde

    # ── 1) locate β_s in the posterior ────────────────────────────────
    prior_samples = (
    beta_df
    .groupby("source")["beta_true"]
    .apply(lambda s: s.values)       # dict: {source → 1-D array}
    .to_dict()
    )
    beta_vars   = [v for v in idata.posterior.data_vars
                   if re.fullmatch(r"beta_\d+", v)]
    sources_avl = sorted(int(v.split("_")[1]) for v in beta_vars)
    if not sources_avl:
        raise ValueError("No beta_* variables found in idata.posterior")

    # ── 2) KDEs for prior & posterior samples ─────────────────────────
    prior_kde  = {}
    post_kde   = {}
    sample_rng = {}

    for s in sources_avl:
        if s not in prior_samples:
            raise KeyError(f"prior_samples missing key {s}")

        # prior KDE
        prior_draws = np.asarray(prior_samples[s]).ravel()
        prior_kde[s] = gaussian_kde(prior_draws, bw_method=kde_bw)

        # posterior KDE
        post_draws  = idata.posterior[f"beta_{s}"].values.ravel()
        post_kde[s] = gaussian_kde(post_draws,  bw_method=kde_bw)

        sample_rng[s] = (min(prior_draws.min(), post_draws.min()),
                         max(prior_draws.max(), post_draws.max()))

    # ── 3) plotting ───────────────────────────────────────────────────
    n      = len(sources_avl)
    ncols  = 2
    nrows  = int(np.ceil(n / ncols))
    fig, axes = plt.subplots(nrows, ncols, figsize=(10, 4 * nrows))
    axes   = axes.flatten()

    for idx, s in enumerate(sources_avl):
        xmin, xmax = sample_rng[s]
        xgrid      = np.linspace(xmin, xmax, xgrid_len)

        p_prior = prior_kde[s](xgrid)
        p_post  = post_kde[s](xgrid)

        # ---- x-axis trimming by density_cut --------------------------
        mask = np.maximum(p_prior, p_post) > density_cut
        if mask.any():
            axes[idx].set_xlim(xgrid[mask].min(), xgrid[mask].max())

        # ---- draw curves --------------------------------------------
        axes[idx].plot(xgrid, p_prior, label="Prior (KDE)")
        axes[idx].plot(xgrid, p_post,  label="Posterior (KDE)")

        axes[idx].set_title(f"Source {s}")
        axes[idx].set_xlabel(rf"$\beta_{{{s}}}$")
        axes[idx].set_ylabel("Density")
        axes[idx].grid(True)
        axes[idx].legend()

    for ax in axes[n:]:
        ax.axis("off")

    fig.tight_layout()
    if save_to is not None:
        fig.savefig(save_to, dpi=300, bbox_inches="tight")
        plt.close(fig)
    else:
        plt.show()

def kl_prior_posterior_beta(
    idata,
    beta_df: pd.DataFrame | None = None,  # prior samples
    *,
    posterior_bins: int = 200,    # size of a *shared* equally–spaced grid
    eps: float = 1e-12,
):
    """
    KL(  posterior  ||  prior )   for each beta_s, where **both** densities are
    estimated from Monte-Carlo draws (prior draws you supply + posterior draws
    inside *idata*).

    Parameters
    ----------
    idata : arviz.InferenceData
        Must contain posterior variables named  beta_<s>.
    prior_samples : dict {source -> 1-D array of draws}
        The raw β draws representing the prior for every source.
        Keys must match the integers “s” in beta_<s>.
    posterior_bins : int
        Number of equally spaced histogram bins used for *both* densities.
    eps : float
        Tiny value added to each density to avoid log(0).

    Returns
    -------
    kl_dict : dict {source : KL divergence (float)}
    """
    prior_samples = (
    beta_df
    .groupby("source")["beta_true"]
    .apply(lambda s: s.values)
    .to_dict()
    )

    # 1) which β_s exist in the posterior?
    beta_vars   = [v for v in idata.posterior.data_vars if re.fullmatch(r"beta_\d+", v)]
    sources_avl = sorted(int(v.split("_")[1]) for v in beta_vars)
    if not sources_avl:
        raise ValueError("No beta_* variables found in idata.posterior")

    kl_dict = {}

    for s in sources_avl:
        if s not in prior_samples:
            raise KeyError(f"prior_samples missing key {s}")

        # ----- sample arrays --------------------------------------------
        prior_draws = np.asarray(prior_samples[s]).ravel()
        post_draws  = idata.posterior[f"beta_{s}"].values.ravel()

        # ----- shared histogram grid ------------------------------------
        xmin = min(prior_draws.min(), post_draws.min())
        xmax = max(prior_draws.max(), post_draws.max())
        edges = np.linspace(xmin, xmax, posterior_bins + 1)
        widths = np.diff(edges)                # Δx per bin
        mids   = 0.5 * (edges[:-1] + edges[1:])

        # ----- densities (area = 1) -------------------------------------
        prior_hist, _ = np.histogram(prior_draws, bins=edges, density=True)
        post_hist,  _ = np.histogram(post_draws,  bins=edges, density=True)

        # ----- KL integral  Σ p_post * log(p_post / p_prior) * Δx --------
        p_post  = post_hist  + eps
        p_prior = prior_hist + eps
        kl      = np.sum(widths * p_post * np.log(p_post / p_prior))

        kl_dict[s] = kl

    return kl_dict

def summarise_beta_prior_posterior(
    idata,
    beta_df: pd.DataFrame | None = None,
    *,
    kl_dict: dict[int, float] | None = None,
    experiment_label: int | str = 1,
):
    """
    Summarise PRIOR vs POSTERIOR for each β_s.

    Parameters
    ----------
    idata : arviz.InferenceData
        Must contain posterior variables named  beta_<s>.
    prior_samples : dict {source → 1-D array of draws}
        Monte-Carlo draws representing the prior for every source.
    kl_dict : dict {source → KL divergence}, optional
        If you have already computed KL( post || prior ) you can pass it here
        so the values are copied into the output table.  Missing keys get NaN.
    experiment_label : hashable
        Copied verbatim into the “experiment” column so you can concatenate
        results from many runs.

    Returns
    -------
    stats_df : pandas.DataFrame
        Columns:
            experiment  source
            prior_mean  prior_median
            post_mean   post_median
            mean_error  median_error
            kl_divergence
            post_ci_lower  post_ci_upper
            ci_covers_prior_mean   (bool)
    """
    prior_samples = (
    beta_df.groupby("source")["beta_true"]
           .apply(lambda s: s.values)       # dict {source → draws}
           .to_dict()
    )

    # ── 1) locate β_s in the posterior ──────────────────────────────────
    beta_vars   = [v for v in idata.posterior.data_vars if re.fullmatch(r"beta_\d+", v)]
    sources_avl = sorted(int(v.split("_")[1]) for v in beta_vars)
    if not sources_avl:
        raise ValueError("No beta_* variables found in idata.posterior")

    if kl_dict is None:
        kl_dict = {}

    records = []

    # 2 ── per-source summary
    for s in sources_avl:
        if s not in prior_samples:
            raise KeyError(f"prior_samples missing key {s}")

        prior_draws  = np.asarray(prior_samples[s]).ravel()
        prior_mean   = float(np.mean(prior_draws))
        prior_median = float(np.median(prior_draws))

        post_draws   = idata.posterior[f"beta_{s}"].values.ravel()
        post_mean    = float(np.mean(post_draws))
        post_median  = float(np.median(post_draws))
        ci_lower, ci_upper = np.percentile(post_draws, [2.5, 97.5])   # ← call via np

        records.append(dict(
            experiment           = experiment_label,
            source               = f"beta{s}",
            prior_mean           = prior_mean,
            prior_median         = prior_median,
            post_mean            = post_mean,
            post_median          = post_median,
            mean_error           = prior_mean - post_mean,
            median_error         = prior_median - post_median,
            kl_divergence        = kl_dict.get(s, np.nan),
            post_ci_lower        = ci_lower,
            post_ci_upper        = ci_upper,
            ci_covers_prior_mean = ci_lower <= prior_mean <= ci_upper,
        ))

    return pd.DataFrame.from_records(records)






,experiment,source,prior_mean,prior_median,post_mean,post_median,mean_error,median_error,kl_divergence,post_ci_lower,post_ci_upper,ci_covers_prior_mean
0,rep1,beta1,1.955559,1.954323,2.006640,2.000830,-0.051081,-0.046507,26.840168,1.911594,2.121373,True
1,rep1,beta2,2.064780,2.064428,2.104604,2.111372,-0.039824,-0.046944,27.471347,2.020283,2.161230,True
2,rep1,beta3,1.867465,1.876424,1.946909,1.938257,-0.079444,-0.061833,27.608923,1.895246,2.039067,False
3,rep1,beta4,1.995572,1.978213,1.926708,1.923170,0.068864,0.055044,27.481497,1.877662,1.990490,False
4,rep1,beta5,1.941855,1.955096,1.973095,1.961325,-0.031240,-0.006229,26.841759,1.903849,2.096841,True
5,rep1,beta6,1.938002,1.934889,1.913469,1.916012,0.024532,0.018877,28.244644,1.850856,1.968141,True
6,rep1,beta7,2.122200,2.106181,2.126976,2.124220,-0.004777,-0.018040,26.510269,2.069363,2.196249,True
7,rep1,beta8,1.873554,1.867908,1.827984,1.832803,0.045569,0.035105,27.361624,1.713374,1.926672,True


## Profile Likelihood for Poisson Regression

For each fixed value of $\beta$ on grid, the log-likelihood for a single observation is

$$
\ell_i(\beta)
= \log p(y_i \mid x_i, \beta)
= y_i \,\bigl(\beta\,x_i\bigr)
- \exp\!\bigl(\beta\,x_i\bigr)
- \log\bigl(y_i!\bigr).
$$

Since $\beta$ is the only parameter, the **profile log-likelihood** is just the sum over all \(n\) observations:

$$
\ell(\beta)
= \sum_{i=1}^n \ell_i(\beta)
= \sum_{i=1}^n \Bigl[y_i(\beta\,x_i) \;-\; e^{\beta\,x_i} \;-\; \log(y_i!)\Bigr].
$$

In [4]:

def simulate_profile_likelihoods_poisson(
    K: int = 5,
    S: int = 8,
    O: int = 8,
    n_obs: int = 100,
    *,
    beta_mean: np.ndarray | None = None,
    beta_sds:  np.ndarray | None = None,
    true_pis:  np.ndarray | None = None,
    grid:      np.ndarray | None = None,
    seed: int = 0,
):
    """
    Simulate profile-likelihood curves for a K-component Poisson-regression mixture
    on every (source, outcome) pair **and save the latent β actually used**.

    Returns
    -------
    df_sim   : DataFrame
        Columns ['source','outcome','point','value']  (log-lik values).
    true_pis : ndarray, shape (S, K)
        Mixture weights used for each source.
    beta_df  : DataFrame
        Columns ['source','outcome','beta_true']  — the latent β for each pair.
    """
    rng = np.random.default_rng(seed)

    # ── 1) hyper-priors for the components ───────────────────────────────
    if beta_mean is None:
        beta_mean = np.linspace(-1, 1, K)
    if beta_sds is None:
        beta_sds = np.full(K, 0.15)

    beta_mean = np.asarray(beta_mean, dtype=float)
    beta_sds  = np.asarray(beta_sds,  dtype=float)
    if beta_mean.shape != (K,) or beta_sds.shape != (K,):
        raise ValueError("beta_mean and beta_sds must be length-K arrays")

    # ── 2) grid for profile likelihoods ──────────────────────────────────
    if grid is None:
        raise ValueError("Provide a 1-D array of grid points via `grid`")
    grid = np.asarray(grid, dtype=float)

    # ── 3) Dirichlet mixture weights per source ──────────────────────────
    if true_pis is None:
        true_pis = rng.dirichlet(np.ones(K) * 2.0, size=S)  # shape (S, K)

    # ── 4) simulation loop ───────────────────────────────────────────────
    recs      = []        # for profile-likelihood rows
    beta_recs = []        # for true β table

    for s in range(S):                  # zero-based 0..S-1
        pi_s = true_pis[s]

        for o in range(O):              # zero-based 0..O-1
            # 4-1) latent component for each observation
            z      = rng.choice(K, size=n_obs, p=pi_s)
            beta_i = rng.normal(loc=beta_mean[z], scale=beta_sds[z])

            # save ONE β per (s,o): here we take the mean of individual β_i
            beta_true = beta_i.mean()
            beta_recs.append(dict(source=s + 1, outcome=o + 1, beta_true=beta_true))

            # 4-2) x-values & Poisson responses
            x_i = rng.uniform(-1, 1, size=n_obs)
            lam = np.exp(beta_i * x_i)
            y_i = rng.poisson(lam)

            # 4-3) profile log-likelihood on grid
            for b in grid:
                logp = y_i * (b * x_i) - np.exp(b * x_i) - gammaln(y_i + 1)
                recs.append(
                    dict(source=s + 1, outcome=o + 1, point=b, value=logp.sum())
                )

    df_sim  = pd.DataFrame.from_records(recs)
    beta_df = pd.DataFrame.from_records(beta_recs)

    return df_sim, true_pis, beta_df

grid_pts   = np.linspace(-10, 10, 100)
#mean draw from gaussian with mean 1 and sd 0.15
beta_mean = rng.normal(loc=2.0, scale=3.0, size=5)
beta_sds = np.abs(rng.normal(loc=0.0, scale=1.0, size=5))

df_sim, true_pis, beta_df = simulate_profile_likelihoods_poisson(
    K=5, S=8, O=8, n_obs=100,
    beta_mean=beta_mean,
    beta_sds=beta_sds,
    grid=grid_pts,
    seed=42
)

df_sim.to_csv("simulated_profileLikelihoods.csv", index=False)


## Profile Likelihood for Linear Regression

For each outcome $o=1,\dots,O$ and observation $i=1,\dots,n$:

\begin{aligned}
z_{s,o,i} &\sim \mathrm{Categorical}(\pi_s),\\
\beta_{\,z_{s,o,i}} &\sim \mathcal{N}(\mu_{z_{s,o,i}},\,\sigma_{z_{s,o,i}}),\\
x_{s,o,i} &\sim \mathrm{Uniform}(-1,1),\\
y_{s,o,i} &= \beta_{\,z_{s,o,i}}\,x_{s,o,i} + \varepsilon_{s,o,i},\quad
\varepsilon_{s,o,i}\sim\mathcal{N}(0,\sigma^2).
\end{aligned}

At each candidate $b$ in the grid:
$$
\ell_{s,o}(b)
=
\sum_{i=1}^n \log \bigl[\mathcal{N}(y_{s,o,i}\mid b\,x_{s,o,i},\,\sigma^2)\bigr]
=
-\frac{n}{2}\log(2\pi\sigma^2)
\;-\;
\frac{1}{2\sigma^2}\sum_{i=1}^n\bigl(y_{s,o,i}-b\,x_{s,o,i}\bigr)^2.
$$

In [5]:
def simulate_profile_likelihoods_linreg(
    K: int = 5,
    S: int = 8,
    O: int = 8,
    n_obs: int = 100,
    *,
    beta_mean: np.ndarray | None = None,
    beta_sds:  np.ndarray | None = None,
    true_pis:  np.ndarray | None = None,
    grid: np.ndarray | None = None,
    sigma_noise: float = 1.0,
    seed: int = 0,
):
    """
    Linear-regression analogue to your Poisson simulator, now ALSO returning a
    table of the latent β actually used for every (source, outcome) pair.

    Returns
    -------
    df_sim   : DataFrame  ['source','outcome','point','value']  (log-lik).
    true_pis : ndarray    shape (S, K)   – mixture weights.
    beta_df  : DataFrame  ['source','outcome','beta_true']      – NEW.
    """
    rng = np.random.default_rng(seed)

    # ── 1) component means / sds ──────────────────────────────────────────
    if beta_mean is None:
        beta_mean = np.linspace(-1, 1, K)
    if beta_sds is None:
        beta_sds = np.full(K, 0.15)
    beta_mean = np.asarray(beta_mean, dtype=float)
    beta_sds  = np.asarray(beta_sds,  dtype=float)
    if beta_mean.shape != (K,) or beta_sds.shape != (K,):
        raise ValueError("`beta_mean` and `beta_sds` must be length-K arrays")

    # ── 2) grid of candidate slopes ──────────────────────────────────────
    if grid is None:
        raise ValueError("Supply a 1-D array of grid points via `grid`.")
    grid = np.asarray(grid, dtype=float)

    # ── 3) Dirichlet weights per source ──────────────────────────────────
    if true_pis is None:
        true_pis = rng.dirichlet(np.full(K, 2.0), size=S)   # (S, K)

    # ── 4) simulate & profile-likelihood evaluation ─────────────────────
    const_term = -0.5 * n_obs * np.log(2 * np.pi * sigma_noise**2)

    recs      = []   # profile-likelihood rows
    beta_recs = []   # true β per (s,o)

    for s in range(S):                               # zero-based
        pi_s = true_pis[s]

        for o in range(O):                           # zero-based
            # 4-1) latent component + β for each observation
            z       = rng.choice(K, size=n_obs, p=pi_s)
            beta_i  = rng.normal(loc=beta_mean[z], scale=beta_sds[z])

            # save ONE representative β  (mean of draws works fine)
            beta_recs.append(
                dict(source=s + 1, outcome=o + 1, beta_true=beta_i.mean())
            )

            # 4-2) covariate & response
            x_i = rng.uniform(-1, 1, size=n_obs)
            y_i = beta_i * x_i + rng.normal(0.0, sigma_noise, size=n_obs)

            # 4-3) profile log-likelihood over the grid
            for b in grid:
                resid_sq = (y_i - b * x_i) ** 2
                ll       = const_term - 0.5 * resid_sq.sum() / sigma_noise**2
                recs.append(
                    dict(source=s + 1, outcome=o + 1, point=b, value=ll)
                )

    df_sim  = pd.DataFrame.from_records(recs)
    beta_df = pd.DataFrame.from_records(beta_recs)

    return df_sim, true_pis, beta_df

grid_pts   = np.linspace(-10, 10, 100)
beta_mean  = rng.normal(loc=1.5, scale=0.7, size=5)
beta_sds = np.abs(rng.normal(loc=0.0, scale=1.0, size=5))

df_sim, pis, beta_df = simulate_profile_likelihoods_linreg(
    K=5, S=8, O=8, n_obs=120,
    beta_mean=beta_mean,
    beta_sds=beta_sds,
    grid=grid_pts,
    sigma_noise=0.8,
    seed=2025
)


## Profile Likelihood for Binomial Regression

For each outcome $o = 1,\dots,O$ and observation $i = 1,\dots,n$:

$$
\begin{aligned}
z_{s,o,i} &\sim \mathrm{Categorical}(\pi_s),\\
\mu_{z_{s,o,i}} &\sim \mathcal{N}(2,9) ,\\
\sigma_{z_{s,o,i}} &\sim \mathrm{HalfNorm}(10.0),\\
b_{z_{s,o,i}} &\sim \mathcal{N}(\mu_{z_{s,o,i}},\,\sigma_{z_{s,o,i}}),\\
x_{s,o,i} &\sim \mathrm{Uniform}(-1,1),\\
p_{s,o,i} &= \sigma\!\bigl(\beta_{z_{s,o,i}}\,x_{s,o,i}\bigr),
\qquad
\sigma(u)=\frac{1}{1+e^{-u}},\\
y_{s,o,i} &\sim \mathrm{Binomial}\!\bigl(n_{s,o,i},\,p_{s,o,i}\bigr).
\end{aligned}
$$

where $n_{s,o,i}$ is the number of trials (often $n_{s,o,i}=1$ for Bernoulli data).

---

### Profile log-likelihood at candidate slope $b$

$$
\ell_{s,o}(b)
=
\sum_{i=1}^n
\log\!\Bigl[
\mathrm{Binomial}\!\bigl(
y_{s,o,i}\mid n_{s,o,i},\,\sigma(b_{s,o,i}\,x_{s,o,i})
\bigr)
\Bigr]
=
\sum_{i=1}^n
\left\{
y_{s,o,i}\,\log\sigma(b_{s,o,i}\,x_{s,o,i})
+\bigl(n_{s,o,i}-y_{s,o,i}\bigr)\,\log\!\bigl[1-\sigma(b_{s,o,i}\,x_{s,o,i})\bigr]
+\log\binom{n_{s,o,i}}{y_{s,o,i}}
\right\}.
$$

(The combinatorial term $\log\binom{n_{s,o,i}}{y_{s,o,i}}$ is constant in $b$ and can be omitted when profiling.)



In [6]:

def simulate_profile_likelihoods_logistic(
    K: int = 5,
    S: int = 8,
    O: int = 8,
    n_obs: int = 100,
    *,
    beta_mean: np.ndarray | None = None,
    beta_sds:  np.ndarray | None = None,
    true_pis:  np.ndarray | None = None,
    grid:      np.ndarray | None = None,
    seed: int = 0,
):
    """
    Simulate profile-likelihood curves for a K-component *logistic-regression*
    mixture **and keep the ground-truth β for every (source, outcome).**

    Returns
    -------
    df_sim   : DataFrame   ['source','outcome','point','value']  (log-liks)
    true_pis : ndarray     shape (S, K)  – mixture weights used
    beta_df  : DataFrame   ['source','outcome','beta_true']      – NEW
    """
    rng = np.random.default_rng(seed)

    # ── 1) component means / sds ─────────────────────────────────────────
    if beta_mean is None:
        beta_mean = np.linspace(-1, 1, K)
    if beta_sds is None:
        beta_sds = np.full(K, 0.15)
    beta_mean = np.asarray(beta_mean, dtype=float)
    beta_sds  = np.asarray(beta_sds,  dtype=float)
    if beta_mean.shape != (K,) or beta_sds.shape != (K,):
        raise ValueError("beta_mean and beta_sds must be length-K arrays")

    # ── 2) grid of candidate β values ───────────────────────────────────
    if grid is None:
        raise ValueError("Provide a 1-D array of grid points via `grid`")
    grid = np.asarray(grid, dtype=float)

    # ── 3) Dirichlet weights per source ─────────────────────────────────
    if true_pis is None:
        true_pis = rng.dirichlet(np.ones(K) * 2.0, size=S)    # (S, K)

    # ── 4) simulate & evaluate profile log-likelihoods ─────────────────
    recs      = []   # profile-likelihood rows
    beta_recs = []   # table of true β’s

    for s in range(S):                              # zero-based 0..S-1
        pi_s = true_pis[s]

        for o in range(O):                          # zero-based 0..O-1
            # 4-1) latent component & β draw per observation
            z       = rng.choice(K, size=n_obs, p=pi_s)
            beta_i  = rng.normal(loc=beta_mean[z], scale=beta_sds[z])

            # save ONE representative β  (mean of draws works fine)
            beta_recs.append(
                dict(source=s + 1, outcome=o + 1, beta_true=beta_i.mean())
            )

            # 4-2) covariate & Bernoulli responses
            x_i     = rng.uniform(-1, 1, size=n_obs)
            logit_p = beta_i * x_i
            p_i     = 1.0 / (1.0 + np.exp(-logit_p))
            y_i     = rng.binomial(1, p_i)

            # 4-3) profile log-likelihood over the grid
            for b in grid:
                bx   = b * x_i
                ll   = (y_i * bx - np.log1p(np.exp(bx))).sum()
                recs.append(
                    dict(source=s + 1, outcome=o + 1, point=b, value=ll)
                )

    df_sim  = pd.DataFrame.from_records(recs)
    beta_df = pd.DataFrame.from_records(beta_recs)
    return df_sim, true_pis, beta_df

rng = np.random.default_rng(42)

grid_pts  = np.linspace(-10, 10, 100)
beta_mean = rng.normal(loc=2.0,  scale=3.0, size=5)
beta_sds  = np.abs(rng.normal(loc=0.0, scale=1.0, size=5))

df_sim, true_pis, beta_df = simulate_profile_likelihoods_logistic(
    K=5, S=8, O=8, n_obs=100,
    beta_mean=beta_mean,
    beta_sds=beta_sds,
    grid=grid_pts,
    seed=42
)

print(df_sim.head())
print("Mixture weights for each source:\n", true_pis)



   source  outcome      point       value
0       1        1 -10.000000 -286.691449
1       1        1  -9.797980 -281.255815
2       1        1  -9.595960 -275.827702
3       1        1  -9.393939 -270.407580
4       1        1  -9.191919 -264.995952
Mixture weights for each source:
 [[0.182076   0.2467942  0.15991495 0.14319025 0.2680246 ]
 [0.16226678 0.21711215 0.20262703 0.2848007  0.13319334]
 [0.35248377 0.10889546 0.22777007 0.2097213  0.10112939]
 [0.31192199 0.18316285 0.09769391 0.33921295 0.06800829]
 [0.08925351 0.17176974 0.28885723 0.25510165 0.19501787]
 [0.1023434  0.28595577 0.3380555  0.19739144 0.07625389]
 [0.19963775 0.2925956  0.13294668 0.32160435 0.05321562]
 [0.08485659 0.21323099 0.1347426  0.29983717 0.26733264]]


In [10]:
N_sources = 8
source_outcome_data = {}
for s in range(1, N_sources+1):
    df_s = df_sim[df_sim["source"]==s]
    outcome_list = []
    for o in sorted(df_s["outcome"].unique()):
        arr = df_s[df_s["outcome"]==o][["point","value"]].to_numpy()
        outcome_list.append(jnp.array(arr))
    source_outcome_data[s] = outcome_list

num_outcomes_dict = {s: len(source_outcome_data[s])
                     for s in range(2, N_sources+1)}

## HDP Model

In [7]:
def stick_breaking(beta):
    rem = jnp.concatenate([jnp.array([1.]), jnp.cumprod(1-beta)[:-1]])
    return beta * rem

def reparameterize(pi):
    return pi / jnp.sum(pi)

def custom_loglike(beta, outcome_data_list):
    total_ll = 0.
    for outcome_data in outcome_data_list:
        x_vals = outcome_data[:,0]
        loglike_vals = outcome_data[:,1]
        total_ll += jnp.interp(beta, x_vals, loglike_vals)
    return total_ll

def HDP_model(source_outcome_data, num_outcomes_dict, N_sources, k, data_point_mean):
    gamma = numpyro.sample("gamma", dist.Gamma(1.0,5.0))
    alpha0 = numpyro.sample("alpha0", dist.Gamma(1.0,5.0))
    beta_tilt = numpyro.sample("beta_tilt", dist.Beta(1.,gamma).expand([k]))
    beta = stick_breaking(beta_tilt)
    beta_cum = jnp.cumsum(beta)
    pi_tilt = numpyro.sample(
        "pi_tilt",
        dist.Beta(alpha0*beta, alpha0*(1-beta_cum))
        .expand([N_sources,k])
    )
    pi_norms = []
    for s in range(N_sources):
        pi_s = stick_breaking(pi_tilt[s])
        pi_norms.append(reparameterize(pi_s))
    mu = numpyro.sample("mu", dist.Normal(0,10.).expand([k]))
    sigma = numpyro.sample("sigma", dist.HalfNormal(10.).expand([k]))
    for s in range(1, N_sources+1):
        mix = dist.MixtureSameFamily(
            dist.Categorical(probs=pi_norms[s-1]),
            dist.Normal(loc=mu, scale=sigma)
        )
        beta_s = numpyro.sample(f"beta_{s}", mix)
        grid_min = jnp.min(source_outcome_data[s][0][:,0])
        grid_max = jnp.max(source_outcome_data[s][-1][:,0])
        beta_clipped = jnp.clip(beta_s, grid_min, grid_max)
        ll = custom_loglike(beta_clipped, source_outcome_data[s])
        numpyro.factor(f"loglike_{s}", ll)

In [9]:
import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist


# --- keep your helper functions ------------------------------------------------
def stick_breaking(beta):
    portion_remaining = jnp.concatenate(
        [jnp.array([1.]), jnp.cumprod(1. - beta)[:-1]]
    )
    return beta * portion_remaining


def reparameterize(pi):
    return pi / pi.sum()

def custom_loglike(beta, outcome_data_list):
    total_ll = 0.
    for outcome_data in outcome_data_list:
        x_vals = outcome_data[:,0]
        loglike_vals = outcome_data[:,1]
        total_ll += jnp.interp(beta, x_vals, loglike_vals)
    return total_ll


# ------------------------------------------------------------------------------
def HDP_model(source_outcome_data,
              num_outcomes_dict,
              N_sources,
              k,
              data_point_mean):
    gamma = numpyro.sample("gamma", dist.Gamma(1.0, 5.0))
    alpha0 = numpyro.sample("alpha0", dist.Gamma(1.0, 5.0))

    # ── global stick-breaking weights ──────────────────────────────────────────
    beta_tilt = numpyro.sample("beta_tilt", dist.Beta(1., gamma).expand([k]))
    beta = stick_breaking(beta_tilt)
    beta_cum = jnp.cumsum(beta)

    # ── source–specific, *unnormalised* sticks ────────────────────────────────
    pi_tilt = numpyro.sample(
        "pi_tilt",
        dist.Beta(alpha0 * beta, alpha0 * (1. - beta_cum)).expand([N_sources, k])
    )  # shape (N_sources, k)

    # ── compute *normalised* mixture weights and **record them** ──────────────
    #     → appear as `pi_norm` in the trace / summary.
    def _to_pi_norm(pi_row):
        return reparameterize(stick_breaking(pi_row))  # (k,)

    pi_norm = jax.vmap(_to_pi_norm)(pi_tilt)  # (N_sources, k)
    numpyro.deterministic("pi_norm", pi_norm)  # <<–– archived!

    # ── remaining model parts (unchanged) ─────────────────────────────────────
    mu = numpyro.sample("mu", dist.Normal(0., 10.).expand([k]))
    sigma = numpyro.sample("sigma", dist.HalfNormal(10.).expand([k]))

    for s in range(1, N_sources + 1):
        mix = dist.MixtureSameFamily(
            dist.Categorical(probs=pi_norm[s - 1]),
            dist.Normal(loc=mu, scale=sigma),
        )
        beta_s = numpyro.sample(f"beta_{s}", mix)

        grid_min = jnp.min(source_outcome_data[s][0][:, 0])
        grid_max = jnp.max(source_outcome_data[s][-1][:, 0])
        beta_clipped = jnp.clip(beta_s, grid_min, grid_max)

        ll = custom_loglike(beta_clipped, source_outcome_data[s])
        numpyro.factor(f"loglike_{s}", ll)

In [28]:
# 5) Run MCMC just as before
rng_key = random.PRNGKey(0)
data_point_mean = df_sim["point"].mean()
k = 5

nuts = NUTS(HDP_model)
mcmc = MCMC(nuts, num_warmup=5000, num_samples=20000)
mcmc.run(
    rng_key,
    source_outcome_data=source_outcome_data,
    num_outcomes_dict=num_outcomes_dict,
    N_sources=N_sources,
    k=k,
    data_point_mean=data_point_mean
)

mcmc.print_summary()
idata = az.from_numpyro(mcmc)

sample: 100%|██████████| 25000/25000 [01:29<00:00, 280.01it/s, 30 steps of size 5.31e-02. acc. prob=0.37] 



                  mean       std    median      5.0%     95.0%     n_eff     r_hat
      alpha0      0.95      0.33      0.88      0.38      1.42    241.85      1.00
      beta_1      0.13      0.12      0.13     -0.07      0.34    157.68      1.00
      beta_2      1.31      0.13      1.31      1.08      1.52    310.35      1.00
      beta_3      1.56      0.14      1.55      1.32      1.78    444.49      1.01
      beta_4      1.59      0.14      1.59      1.35      1.84    268.66      1.00
      beta_5      0.87      0.12      0.87      0.69      1.07    352.18      1.00
      beta_6      1.12      0.13      1.11      0.90      1.32    177.72      1.00
      beta_7      1.43      0.14      1.43      1.23      1.66    274.69      1.00
      beta_8      0.58      0.12      0.58      0.38      0.77    161.69      1.02
beta_tilt[0]      0.29      0.22      0.19      0.03      0.64     27.86      1.11
beta_tilt[1]      0.29      0.21      0.21      0.03      0.63     49.79      1.05
bet

In [10]:
def store_beta_posteriors(
        idata,
        df_sim: pd.DataFrame,
        *,
        output_folder: str | Path,
        file_ext: str = "npz",
        compress: bool = True,
        regex: str = r"beta_\d+",
) -> dict[str, dict[str, np.ndarray]]:
    """
    Persist posterior samples of β parameters plus their empirical priors.

    For each posterior variable that matches *regex*
    (default pattern ``beta_<number>``) the function

    1. extracts all posterior draws and flattens them to 1-D;
    2. selects the slice of *df_sim* that belongs to the same *source*
       (the integer following the underscore);
    3. writes a single file

           beta_<s>.npz   (or .npy)

       with three NumPy arrays:
           • samples – posterior draws            (shape = [n_draws])
           • grid    – grid points used in prior  (shape = [m])
           • loglik  – log-likelihood on that grid (shape = [m])

    Parameters
    ----------
    idata : arviz.InferenceData
        Posterior container produced by NumPyro / ArviZ.
    df_sim : pandas.DataFrame
        Columns ['source','outcome','point','value']; *value* is a
        log-likelihood.  Log-likelihoods for identical (source, point)
        pairs are summed before being stored.
    output_folder : str | pathlib.Path
        Where the files are written (created if missing).
    file_ext : {"npz","npy"}, default "npz"
        Storage format.  If "npz" you may enable compressed archives
        via *compress=True* (default).
    compress : bool, default True
        Compression flag for the “npz” format.
    regex : str, default r"beta_\\d+"
        Pattern that selects β variables inside *idata.posterior*.

    Returns
    -------
    dict
        A mapping ``{ "beta_<s>" : { "samples":…, "grid":…, "loglik":… }, … }``
        matching what has been written to disk.
    """

    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)

    beta_vars = [v for v in idata.posterior.data_vars if re.fullmatch(regex, v)]
    if not beta_vars:
        raise ValueError(f"No beta variables match {regex!r}")

    result: dict[str, dict[str, np.ndarray]] = {}

    for var in sorted(beta_vars, key=lambda v: int(v.split("_")[1])):
        s = int(var.split("_")[1])  # source number
        samples = idata.posterior[var].values.reshape(-1)

        # ---- empirical prior slice (summed over outcomes) ----
        mask = df_sim["source"] == s
        if not mask.any():
            raise ValueError(f"df_sim contains no rows for source={s}")

        sub = (
            df_sim.loc[mask, ["point", "value"]]
            .groupby("point", sort=True)["value"]
            .sum()
            .sort_index()
        )
        grid, loglik = sub.index.to_numpy(), sub.values

        bundle = {"samples": samples, "grid": grid, "loglik": loglik}
        result[var] = bundle

        fname = output_folder / f"{var}.{file_ext}"
        if file_ext == "npz":
            (np.savez_compressed if compress else np.savez)(fname, **bundle)
        elif file_ext == "npy":
            np.save(fname, bundle, allow_pickle=True)
        else:
            raise ValueError("file_ext must be 'npz' or 'npy'")

    print(f"Stored {len(result)} β posteriors in {output_folder}")
    return result

def load_beta_posteriors(
        folder: str | Path,
        *,
        file_ext: str = "npz",
        regex: str = r"beta_\d+\."  # automatically completed with file_ext
) -> dict[str, dict[str, np.ndarray]]:
    """
    Read back the objects produced by `store_beta_posteriors`.

    Parameters
    ----------
    folder : str | pathlib.Path
        Directory that contains the saved `beta_<s>.<ext>` files.
    file_ext : {"npz", "npy"}, default "npz"
        Extension that was used when saving.
    regex : str, default r"beta_\\d+\\."
        A prefix for the file-name pattern; the extension is appended
        automatically.  Change only if you saved under a different name
        convention.

    Returns
    -------
    dict
        Mapping ``{ "beta_<s>" : { "samples":…, "grid":…, "loglik":… }, … }``.
    """
    folder = Path(folder)
    if not folder.is_dir():
        raise FileNotFoundError(f"{folder} is not a directory")

    pattern = re.compile(regex + re.escape(file_ext) + r"$")
    output: dict[str, dict[str, np.ndarray]] = {}

    for f in sorted(folder.iterdir()):
        if not pattern.fullmatch(f.name):
            continue
        varname = f.stem  # e.g. "beta_1"
        if file_ext == "npz":
            with np.load(f, allow_pickle=False) as data:
                output[varname] = {
                    "samples": data["samples"],
                    "grid": data["grid"],
                    "loglik": data["loglik"],
                }
        elif file_ext == "npy":
            output[varname] = np.load(f, allow_pickle=True).item()
        else:
            raise ValueError("file_ext must be 'npz' or 'npy'")

    if not output:
        raise RuntimeError(f"No *.{file_ext} files that match {regex} found in {folder}")

    print(f"Loaded {len(output)} β posterior archives from {folder}")
    return output





# largest_mean_pi_norm.py
from __future__ import annotations
import numpy as np
import xarray as xr
from typing import List, Dict, Any


def _as_ndarray(obj) -> np.ndarray:
    """Return obj as a NumPy array, handling xarray objects transparently."""
    if isinstance(obj, (np.ndarray, list, tuple)):
        return np.asarray(obj)
    if isinstance(obj, (xr.DataArray, xr.Dataset)):
        return obj.values
    raise TypeError(
        "Expected NumPy array-like or xarray object for `pis`, "
        f"got {type(obj).__name__}"
    )


def largest_mean_pi_norm(
    idata,
    *,
    pis: Any | None = None,
    pis_var: str = "pis",
) -> List[Dict[str, Any]]:
    """
    For every source s
    ---------------------------------
    1. find the component c that maximises the posterior mean of
       `pi_norm[s, c]`;
    2. also report the *true* largest proportion, i.e.
       max_c pis[s, c].

    Parameters
    ----------
    idata : arviz.InferenceData
        Must contain posterior variable 'pi_norm'.
    pis : array-like or xarray.DataArray, optional
        Ground-truth mixing proportions with shape (n_sources, n_components).
        If not supplied the function tries to read `idata.posterior[pis_var]`.
    pis_var : str
        Name of the variable inside `idata.posterior` that stores `pis`
        (ignored if `pis` argument is provided).

    Returns
    -------
    List[dict]
        One dictionary per source with keys
        • source          – int (0-based)
        • component       – int, index of argmax of mean π
        • mean_value      – float, posterior mean π at that (s, c)
        • true_proportion – float, max_c pis[s, c] if available, else None
    """
    # ── posterior mean of pi_norm ―――――――――――――――――――――――――――――――――――――――
    mean_pi = idata.posterior["pi_norm"].mean(dim=("chain", "draw"))  # (source, component)
    mean_vals = mean_pi.values                                        # ndarray (S, C)

    # best component index for each source
    best_comp_idx = mean_vals.argmax(axis=1)                          # shape (S,)

    # ── fetch / validate true proportions ――――――――――――――――――――――――――――――
    if pis is None:
        if pis_var in idata.posterior:
            pis = idata.posterior[pis_var]
        else:
            pis = None

    pis_vals = None
    if pis is not None:
        pis_vals = _as_ndarray(pis)
        if pis_vals.shape != mean_vals.shape:
            raise ValueError(
                "Shape mismatch between `pis` "
                f"{pis_vals.shape} and pi_norm mean {mean_vals.shape}"
            )

    # ── assemble results ――――――――――――――――――――――――――――――――――――――――――――――
    results = []
    for src, comp in enumerate(best_comp_idx):
        true_prop = None
        if pis_vals is not None:
            true_prop = float(pis_vals[src].max())

        results.append(
            {
                "source": src,
                "component": int(comp),
                "mean_value": float(mean_vals[src, comp]),
                "true_proportion": true_prop,
            }
        )
    return results








In [ ]:
beta_store = store_beta_posteriors(
    idata=idata,
    df_sim=df_sim,
    output_folder="results/beta_archives",
    file_ext="npz",      # or "npy"
    compress=True,
)

beta_data = load_beta_posteriors(
    folder="results/beta_archives",
    file_ext="npz",     # or "npy" if you saved .npy files
)

# Access arrays for source 3
b3 = beta_data["beta_3"]
samples_b3 = b3["samples"]
grid_b3    = b3["grid"]
loglik_b3  = b3["loglik"]

best_per_source = largest_mean_pi_norm(idata, pis=pis)  # if you have one

for rec in best_per_source:
    print(
        f"Source {rec['source']:>2}: "
        f"best component={rec['component']:>2}, "
        f"mean π={rec['mean_value']:.3f}, "
        f"true max π={rec['true_proportion']:.3f}"
    )


In [ ]:
save_beta_density_hist(
    idata,
    output_folder="figures/beta_plots",
    experiment="sim1",          # will create sim1_beta_0.jpg, …
)

In [ ]:
plot_df_prior_vs_posterior(df_sim, idata)

In [ ]:
plot_df_prior_vs_posterior( idata, beta_df)

In [ ]:
kl_vals = kl_prior_posterior_beta(df_sim, idata)
for s, kl in kl_vals.items():
    print(f"Source {s}: KL(posterior‖prior) = {kl:0.4f}")


In [ ]:
stats = summarise_beta_prior_posterior(df_sim, idata,1)
print(stats)

# save to csv
stats.to_csv("beta_prior_posterior_stats.csv", index=False)


In [ ]:
'''
def summarise_beta_prior_posterior_sample(
        df_sim,
        idata,
        experiment: int = 1
):
    """
    For each β_s parameter compute

        • prior sample mean & median (raw `point` column of df_sim)
        • posterior sample mean, median, 95 % CI
        • flag : does the 95 % CI cover the prior sample mean?

    Returns
    -------
    pandas.DataFrame with columns
        source
        prior_mean prior_median
        post_mean post_median
        post_ci_lower post_ci_upper
        ci_covers_prior_mean
    """
    # ── locate β_s in the posterior ───────────────────────────────
    beta_vars = [v for v in idata.posterior.data_vars
                 if re.fullmatch(r"beta_\d+", v)]
    sources_avl = sorted(int(v.split("_")[1]) for v in beta_vars)
    if not sources_avl:
        raise ValueError("No beta_* variables found in idata.posterior")

    rows = []

    for s in sources_avl:
        # ── PRIOR sample statistics (raw grid points) -------------
        prior_samples = df_sim.loc[df_sim["source"] == s, "point"].values
        prior_mean = float(np.mean(prior_samples))
        prior_median = float(np.median(prior_samples))

        # ── POSTERIOR sample statistics ---------------------------
        post_samples = idata.posterior[f"beta_{s}"].values.flatten()
        post_mean = float(np.mean(post_samples))
        post_median = float(np.median(post_samples))
        ci_lower, ci_upper = np.percentile(post_samples, [2.5, 97.5])

        covers = (ci_lower <= prior_mean) and (prior_mean <= ci_upper)

        rows.append(dict(
            experiment=experiment,
            source=s,
            prior_mean=prior_mean,
            prior_median=prior_median,
            post_mean=post_mean,
            post_median=post_median,
            post_ci_lower=ci_lower,
            post_ci_upper=ci_upper,
            ci_covers_prior_mean=covers,
        ))

    return pd.DataFrame(rows)

stats = summarise_beta_prior_posterior(df_sim, idata, 1)
print(stats)
'''

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import jax.random as random
import jax.numpy as jnp
import arviz as az
from numpyro.infer import MCMC, NUTS
from datetime import date



# ------------------------------------------------------------------
#  Main driver
# ------------------------------------------------------------------
def run_hdp_experiments(
    K: int,
    O: int,
    S: int,
    n_obs: int,
    N_sources: int,
    seed: int,
    output_base: str,
    *,
    num_warmup: int = 5000,
    num_samples: int = 20000,
    n_reps: int = 10,                 # ← how many repetitions (k times)
):
    """
    Run the complete HDP simulation / fitting experiment `n_reps` times for
    *each* of the three data-generating families: linear, Poisson, logistic.

    All artefacts are written beneath

        {output_base}/{model_type}/

    where `model_type` ∈ {"linear","poisson","logistic"}.
    A single CSV containing the concatenated summary statistics is produced
    for every model family.

    Parameters
    ----------
    K, O, S, n_obs
        Mixture, outcome & observation sizes to pass through to your
        simulate_profile_likelihoods_* helpers.
    N_sources
        Preset number of sources in the HDP.
    seed
        Global seed.  Repetition-specific seeds are derived from it so that no
        two repetitions ever share the same random stream.
    output_base
        Root directory in which all results will be stored.
    num_warmup, num_samples
        Sampler settings for the NUTS-based MCMC.
    n_reps
        Number of repetitions **per model family**.
    """

    today_str = date.today().isoformat()
    rng_master = np.random.default_rng(seed)

    # Map each model family to its simulator helper ---------------------------
    sim_fns = dict(
        linear   = simulate_profile_likelihoods_linreg,
        poisson  = simulate_profile_likelihoods_poisson,
        logistic = simulate_profile_likelihoods_logistic,
    )

    for model_type, sim_fn in sim_fns.items():
        model_dir = Path(output_base) / model_type
        fig_dir   = model_dir / "figures" / "beta_plots"
        data_dir  = model_dir / "data"
        fig_dir.mkdir(parents=True, exist_ok=True)

        stats_all   = []   # collects one DF per repetition
        kl_all      = []
        proportion_largest = []

        for rep in range(1, n_reps + 1):
            # ----------------------------------------------------------------
            # 1)  Data generation
            # ----------------------------------------------------------------
            rep_seed = rng_master.integers(seed+rep)
            rng_rep  = np.random.default_rng(rep_seed)

            grid_pts  = np.linspace(-10, 10, 100)
            beta_mean = rng_rep.normal(loc=1.5, scale=0.7, size=K)
            beta_sds  = np.abs(rng_rep.normal(loc=0.0, scale=1.0, size=K))

            # Simulator may have family-specific keyword parameters.
            sim_kwargs = dict(
                K=K, S=S, O=O, n_obs=n_obs,
                beta_mean=beta_mean,
                beta_sds=beta_sds,
                grid=grid_pts,
                seed=rep_seed,
            )
            if model_type == "linear":
                sim_kwargs["sigma_noise"] = 0.8

            df_sim, pis_true, df_betas = sim_fn(**sim_kwargs)

            # ----------------------------------------------------------------
            # 2)  Reshape into source-outcome dict expected by HDP_model
            # ----------------------------------------------------------------
            source_outcome_data = {}
            for s in range(1, N_sources + 1):
                df_s = df_sim[df_sim["source"] == s]
                outcome_list = []
                for o in sorted(df_s["outcome"].unique()):
                    arr = df_s[df_s["outcome"] == o][["point", "value"]].to_numpy()
                    outcome_list.append(jnp.array(arr))
                source_outcome_data[s] = outcome_list

            num_outcomes_dict = {s: len(source_outcome_data[s])
                                 for s in range(1, N_sources + 1)}

            # ----------------------------------------------------------------
            # 3)  Run HDP
            # ----------------------------------------------------------------
            rng_key = random.PRNGKey(rep_seed)
            data_point_mean = jnp.asarray(df_sim["point"].mean())

            nuts_kernel = NUTS(HDP_model)
            mcmc = MCMC(nuts_kernel, num_warmup=num_warmup, num_samples=num_samples)
            mcmc.run(
                rng_key,
                source_outcome_data=source_outcome_data,
                num_outcomes_dict=num_outcomes_dict,
                N_sources=N_sources,
                k=K,
                data_point_mean=data_point_mean,
            )
            idata = az.from_numpyro(mcmc)

            store_beta_posteriors(
                idata=idata,
                df_sim=df_sim,
                output_folder=data_dir/f"rep{rep}",
                file_ext="npz",      # or "npy"
                compress=True,
            )

            # ----------------------------------------------------------------
            # 4)  Save betas density histograms
            # ----------------------------------------------------------------
            save_beta_density_hist(
                idata,
                output_folder=str(fig_dir),
                experiment=f"rep{rep}",
            )

            # ----------------------------------------------------------------
            # 5)  Save PPC (prior vs posterior) comparison
            # ----------------------------------------------------------------
            ppc_path = model_dir / f"rep{rep}_ppc.png"
            ppc_obj  = plot_df_prior_vs_posterior(df_sim, idata, save_to=ppc_path)


            if isinstance(ppc_obj, plt.Figure):           # helper returns a Figure
                ppc_obj.savefig(ppc_path, dpi=300, bbox_inches="tight")
                plt.close(ppc_obj)

            elif hasattr(ppc_obj, "figure"):              # helper returns an Axes
                ppc_obj.figure.savefig(ppc_path, dpi=300, bbox_inches="tight")
                plt.close(ppc_obj.figure)

            else:
                # Helper already saved the plot (or user doesn’t need a file).
                # If you still want confirmation:
                print(f"  [info] plot_df_prior_vs_posterior returned {type(ppc_obj)}; "
                      "skipping explicit save.")

            # ----------------------------------------------------------------
            # 6)  KL divergence
            # ----------------------------------------------------------------
            kl_vals = kl_prior_posterior_beta(df_sim, idata)
            kl_vals["rep"] = rep          # keep track of repetition
            kl_all.append(kl_vals)

            # ----------------------------------------------------------------
            # 7)  Summary statistics
            # ----------------------------------------------------------------
            stats = summarise_beta_prior_posterior(df_sim, idata, kl_vals, experiment_label=f"rep{rep}")
            largest_info = largest_mean_pi_norm(idata, pis=pis)
            largest_df   = pd.DataFrame(largest_info).rename(columns={
                "component":      "largest_component",
                "mean_value":     "mean_pi_norm",
                "true_proportion":"true_max_pi",
                "source":         "src_zero",
            })

            # 3 ── create the join key in *stats*  (convert “beta1” → 0, “beta2” → 1, …)
            stats["src_zero"] = stats["source"].str.extract(r"(\d+)").astype(int) - 1

            # 4 ── merge and drop helper key
            stats = (
                stats
                  .merge(largest_df, on="src_zero", how="left")
                  .drop(columns="src_zero")
            )


            stats_all.append(stats)

            print(f"[{model_type}] repetition {rep} finished.")

        # --------------------------------------------------------------------
        #  Persist concatenated results for this model family
        # --------------------------------------------------------------------
        stats_df = pd.concat(stats_all, ignore_index=True)
        fname = model_dir / f"beta_summary_stats_{today_str}.csv"
        stats_df.to_csv(fname, index=False)


        print(f"\nCompleted all {n_reps} repetitions for '{model_type}'. "
              f"Results stored in: {model_dir}\n")

run_hdp_experiments(
    K=5, O=8, S=8, n_obs=120,
    N_sources=8,
    seed=42,
    output_base="results",
    num_warmup=5000,
    num_samples=20000,
    n_reps=100
)


sample:  26%|██▌       | 6432/25000 [00:40<03:12, 96.62it/s, 255 steps of size 1.17e-02. acc. prob=0.91]  